# Kmeans Modelos Seleccionados

## 1. Importar librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.cluster import KMeans
from random import sample
from sklearn.ensemble import IsolationForest
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import shap

#Mostrar todas las columnas de un dataframe
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## 2. Importación de data

In [2]:
#Seleccionar ruta de la data
dataframe = pd.read_csv(r'C:\GIT_Cumplimiento\epic\EPIC003\data\modeling\03_train_v4.csv', encoding='latin')
dataframe.shape

(13516, 13)

## 3. Modelo

In [3]:
#REviso columnas
dataframe.columns

Index(['Unnamed: 0', 'PERIODO', 'CODCLAVECIC', 'NBRCLIORDENANTE', 'SEGMENTO',
       'MTO_TRANSF', 'CTD_OPE', 'FLG_PEP', 'FLG_PROF', 'FLG_PAR', 'FLG_PERFIL',
       'CTDEVAL', 'FLG_PAIS'],
      dtype='object')

### Model 1: Variables modelo estándar (Todas las variables)

In [4]:
#Selecciono variables
ds1=dataframe[['MTO_TRANSF', 'CTD_OPE', 'FLG_PEP', 'FLG_PROF', 'FLG_PERFIL','CTDEVAL', 'FLG_PAIS']]
#Estandarizo variables
data1_cluster = MinMaxScaler().fit_transform(ds1)

#Calculo silueta final  k=18
ki=9
outputKMeans = KMeans(n_clusters = ki, init='k-means++', random_state = 7).fit(data1_cluster)
scoreSilhoutte = metrics.silhouette_score(data1_cluster, outputKMeans.labels_, metric='euclidean')
print(" k=", ki," SC=", scoreSilhoutte)

 k= 9  SC= 0.8670152321782827


In [5]:
#Entreno modelo
clustering1=KMeans(n_clusters=9,init='k-means++', random_state = 7)
clustering1.fit(data1_cluster)
#Asigno etiquetas de cluster como variable
dataframe1=dataframe.copy()
dataframe1['N_Cluster']=clustering1.labels_
#Valido creación de variable con clusters
dataframe1.head(5)

,Unnamed: 0,PERIODO,CODCLAVECIC,NBRCLIORDENANTE,SEGMENTO,MTO_TRANSF,CTD_OPE,FLG_PEP,FLG_PROF,FLG_PAR,FLG_PERFIL,CTDEVAL,FLG_PAIS,N_Cluster
0,0,202206,812,CORTES DE POLAR SANDRA MIRIAN,ENALTA,100000.00,1,0,1,2,0,0,1,1
1,8,202209,1750,CONROY DE MARSANO MARIA TERESA,PRIVADA,17357.00,1,0,0,0,0,15,0,2
2,9,202204,1973,APARCANA HERRERA MARIELA VILMA,CONSUMO,13931.65,1,0,1,0,0,0,0,3
3,12,202204,2249,DANERI PREIS GUSTAVO,ENALTA,13275.00,1,0,1,0,0,5,0,3
4,14,202208,2374,FIEDLER VASQUEZ-MEJIA PAUL,ENALTA,10670.96,2,0,1,0,0,0,0,3


In [18]:
#Valido número de casos por cluster
dataframe1.groupby('N_Cluster').N_Cluster.count().to_clipboard()
dataframe1.groupby('N_Cluster').N_Cluster.count()

N_Cluster
0    3608
1    3064
2    4393
3    1234
4     471
5     290
6      95
7     232
8     129
Name: N_Cluster, dtype: int64

In [19]:
#Perfilamiento media por variable por cluster
dataframe1.groupby('N_Cluster').mean().round(3).to_clipboard()
dataframe1.groupby('N_Cluster').mean().round(3)

,Unnamed: 0,PERIODO,CODCLAVECIC,MTO_TRANSF,CTD_OPE,FLG_PEP,FLG_PROF,FLG_PAR,FLG_PERFIL,CTDEVAL,FLG_PAIS
N_Cluster,,,,,,,,,,,
0,25564.902,202206.501,6952805.826,52728.774,1.486,0.000,0.0,1.905,0.0,0.478,1.000
1,30882.737,202206.570,9288249.652,45374.297,1.569,0.004,1.0,1.955,0.0,0.230,1.000
2,23395.030,202206.552,6191217.597,46826.495,1.609,0.000,0.0,0.000,0.0,1.142,0.000
3,25525.909,202206.522,7236594.302,41916.134,1.812,0.011,1.0,0.000,0.0,1.010,0.000
4,23157.807,202206.561,6216680.195,60486.849,2.357,0.023,0.0,0.000,1.0,1.930,0.000
5,26309.414,202206.493,7145237.734,65790.747,2.424,0.010,0.0,1.838,1.0,0.679,1.000
6,22813.747,202206.389,6050748.105,55697.304,2.547,0.000,1.0,0.000,1.0,0.916,0.000
7,31749.056,202206.591,9503042.690,45540.474,2.522,0.004,1.0,1.927,1.0,0.241,1.000
8,13469.798,202206.574,2120231.504,55328.921,1.426,1.000,0.0,0.783,0.0,8.643,0.419


In [20]:
#Perfilamiento desviacióne estándar por variable por cluster
dataframe1.groupby('N_Cluster').std().round(3).to_clipboard()
dataframe1.groupby('N_Cluster').std().round(3)

,Unnamed: 0,PERIODO,CODCLAVECIC,MTO_TRANSF,CTD_OPE,FLG_PEP,FLG_PROF,FLG_PAR,FLG_PERFIL,CTDEVAL,FLG_PAIS
N_Cluster,,,,,,,,,,,
0,15364.324,1.734,7415052.635,65951.588,1.168,0.000,0.0,0.293,0.0,2.192,0.000
1,14619.484,1.720,8135019.078,58597.452,1.125,0.062,0.0,0.208,0.0,1.266,0.000
2,15646.438,1.720,7248341.176,63349.409,1.240,0.000,0.0,0.000,0.0,3.447,0.000
3,15928.115,1.715,8063699.792,57113.657,1.350,0.102,0.0,0.000,0.0,4.141,0.000
4,15975.060,1.697,7427242.264,77798.666,1.738,0.151,0.0,0.000,0.0,8.577,0.000
5,15116.630,1.738,7151865.923,81340.992,1.648,0.101,0.0,0.369,0.0,2.463,0.000
6,15466.121,1.684,7770165.128,64442.145,1.662,0.000,0.0,0.000,0.0,2.934,0.000
7,13999.014,1.716,7713505.105,51498.517,2.304,0.066,0.0,0.261,0.0,0.849,0.000
8,10863.010,1.815,3085032.510,69393.573,0.778,0.000,0.0,0.952,0.0,28.736,0.495


In [9]:
#Exporto tabla con variable del modelo a excel
dataframe1.to_excel(r'C:\GIT_Cumplimiento\epic\EPIC003\reports\others\KM1_9seg_stair.xlsx',index=False,header=True)

In [10]:
#Guardar el modelo
import pickle
pickle.dump(clustering1, open('KMeans1_9seg_stairs.model', 'wb'))

### Model 2: Variables modelo estándar sin variable CTD_OPE

In [11]:
#Selecciono variables
ds2=dataframe[['MTO_TRANSF', 'CTDEVAL', 'FLG_PEP', 'FLG_PROF', 'FLG_PERFIL','FLG_PAIS']]
#Estandarizo variables
data2_cluster = MinMaxScaler().fit_transform(ds2)
#Selecciono cluster silueta final
ki = 9
#Calculo silueta
outputKMeans = KMeans(n_clusters = ki, init='k-means++', random_state = 7).fit(data2_cluster)
scoreSilhoutte = metrics.silhouette_score(data2_cluster, outputKMeans.labels_, metric='euclidean')
print(" k=", ki," SC=", scoreSilhoutte)

 k= 9  SC= 0.8879059479411482


In [12]:
#Entreno modelo
clustering2=KMeans(n_clusters=9,init='k-means++', random_state = 7)
clustering2.fit(data2_cluster)
#Asigno etiquetas de cluster como variable
dataframe2=dataframe.copy()
dataframe2['N_Cluster']=clustering2.labels_
#Valido creación de variable con clusters
dataframe2.head(5)

,Unnamed: 0,PERIODO,CODCLAVECIC,NBRCLIORDENANTE,SEGMENTO,MTO_TRANSF,CTD_OPE,FLG_PEP,FLG_PROF,FLG_PAR,FLG_PERFIL,CTDEVAL,FLG_PAIS,N_Cluster
0,0,202206,812,CORTES DE POLAR SANDRA MIRIAN,ENALTA,100000.00,1,0,1,2,0,0,1,0
1,8,202209,1750,CONROY DE MARSANO MARIA TERESA,PRIVADA,17357.00,1,0,0,0,0,15,0,1
2,9,202204,1973,APARCANA HERRERA MARIELA VILMA,CONSUMO,13931.65,1,0,1,0,0,0,0,3
3,12,202204,2249,DANERI PREIS GUSTAVO,ENALTA,13275.00,1,0,1,0,0,5,0,3
4,14,202208,2374,FIEDLER VASQUEZ-MEJIA PAUL,ENALTA,10670.96,2,0,1,0,0,0,0,3


In [13]:
#Valido número de casos por cluster
dataframe2.groupby('N_Cluster').N_Cluster.count().to_clipboard()
dataframe2.groupby('N_Cluster').N_Cluster.count()

N_Cluster
0    3064
1    4393
2    3608
3    1234
4     471
5     290
6     232
7     129
8      95
Name: N_Cluster, dtype: int64

In [14]:
#Perfilamiento media por variable por cluster
dataframe2.groupby('N_Cluster').mean().round(3).to_clipboard()
dataframe2.groupby('N_Cluster').mean().round(3)

,Unnamed: 0,PERIODO,CODCLAVECIC,MTO_TRANSF,CTD_OPE,FLG_PEP,FLG_PROF,FLG_PAR,FLG_PERFIL,CTDEVAL,FLG_PAIS
N_Cluster,,,,,,,,,,,
0,30882.737,202206.570,9288249.652,45374.297,1.569,0.004,1.0,1.955,0.0,0.230,1.000
1,23395.030,202206.552,6191217.597,46826.495,1.609,0.000,0.0,0.000,0.0,1.142,0.000
2,25564.902,202206.501,6952805.826,52728.774,1.486,0.000,0.0,1.905,0.0,0.478,1.000
3,25525.909,202206.522,7236594.302,41916.134,1.812,0.011,1.0,0.000,0.0,1.010,0.000
4,23157.807,202206.561,6216680.195,60486.849,2.357,0.023,0.0,0.000,1.0,1.930,0.000
5,26309.414,202206.493,7145237.734,65790.747,2.424,0.010,0.0,1.838,1.0,0.679,1.000
6,31749.056,202206.591,9503042.690,45540.474,2.522,0.004,1.0,1.927,1.0,0.241,1.000
7,13469.798,202206.574,2120231.504,55328.921,1.426,1.000,0.0,0.783,0.0,8.643,0.419
8,22813.747,202206.389,6050748.105,55697.304,2.547,0.000,1.0,0.000,1.0,0.916,0.000


In [15]:
#Perfilamiento desviacióne estándar por variable por cluster
dataframe2.groupby('N_Cluster').std().round(3).to_clipboard()
dataframe2.groupby('N_Cluster').std().round(3)

,Unnamed: 0,PERIODO,CODCLAVECIC,MTO_TRANSF,CTD_OPE,FLG_PEP,FLG_PROF,FLG_PAR,FLG_PERFIL,CTDEVAL,FLG_PAIS
N_Cluster,,,,,,,,,,,
0,14619.484,1.720,8135019.078,58597.452,1.125,0.062,0.0,0.208,0.0,1.266,0.000
1,15646.438,1.720,7248341.176,63349.409,1.240,0.000,0.0,0.000,0.0,3.447,0.000
2,15364.324,1.734,7415052.635,65951.588,1.168,0.000,0.0,0.293,0.0,2.192,0.000
3,15928.115,1.715,8063699.792,57113.657,1.350,0.102,0.0,0.000,0.0,4.141,0.000
4,15975.060,1.697,7427242.264,77798.666,1.738,0.151,0.0,0.000,0.0,8.577,0.000
5,15116.630,1.738,7151865.923,81340.992,1.648,0.101,0.0,0.369,0.0,2.463,0.000
6,13999.014,1.716,7713505.105,51498.517,2.304,0.066,0.0,0.261,0.0,0.849,0.000
7,10863.010,1.815,3085032.510,69393.573,0.778,0.000,0.0,0.952,0.0,28.736,0.495
8,15466.121,1.684,7770165.128,64442.145,1.662,0.000,0.0,0.000,0.0,2.934,0.000


In [16]:
#Exporto tabla con variable del modelo a excel
dataframe2.to_excel(r'C:\GIT_Cumplimiento\epic\EPIC003\reports\others\KM2_9seg_stair.xlsx',index=False,header=True)

In [17]:
#Guardar el modelo
import pickle
pickle.dump(clustering1, open('KMeans2_9seg_stairs.model', 'wb'))